In [1]:
#required Libraries

import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import glob2
import os, fnmatch
from pathlib import Path

# Processing the Videos

In [2]:
#importing file function
def find_files(directory, pattern):
    for root, dirs, files in os.walk(directory):
        for basename in files:
            if fnmatch.fnmatch(basename, pattern):
                filename = os.path.join(root, basename)
                yield filename


In [5]:
file_name = []
for filename in find_files('../Dataset', '*.avi'):
    file_name.append(filename)
    print('Found C source:', filename)

Found C source: ../Dataset/VidTIMIT/mrjo0/sx104.avi
Found C source: ../Dataset/VidTIMIT/mrjo0/si734.avi
Found C source: ../Dataset/VidTIMIT/mrjo0/sx14.avi
Found C source: ../Dataset/VidTIMIT/mrjo0/sa2.avi
Found C source: ../Dataset/VidTIMIT/mrjo0/sa1.avi
Found C source: ../Dataset/VidTIMIT/mrjo0/sx374.avi
Found C source: ../Dataset/VidTIMIT/mrjo0/si1624.avi
Found C source: ../Dataset/VidTIMIT/mrjo0/si1364.avi
Found C source: ../Dataset/VidTIMIT/mrjo0/sx194.avi
Found C source: ../Dataset/VidTIMIT/mrjo0/sx284.avi
Found C source: ../Dataset/VidTIMIT/mrcz0/sx11.avi
Found C source: ../Dataset/VidTIMIT/mrcz0/si911.avi
Found C source: ../Dataset/VidTIMIT/mrcz0/si2171.avi
Found C source: ../Dataset/VidTIMIT/mrcz0/sx101.avi
Found C source: ../Dataset/VidTIMIT/mrcz0/sx371.avi
Found C source: ../Dataset/VidTIMIT/mrcz0/si1541.avi
Found C source: ../Dataset/VidTIMIT/mrcz0/sa2.avi
Found C source: ../Dataset/VidTIMIT/mrcz0/sa1.avi
Found C source: ../Dataset/VidTIMIT/mrcz0/sx191.avi
Found C source: ..

Found C source: ../Dataset/DeepfakeTIMIT/lower_quality/fadg0/sa1-video-fram1.avi
Found C source: ../Dataset/DeepfakeTIMIT/lower_quality/fadg0/sx19-video-fram1.avi
Found C source: ../Dataset/DeepfakeTIMIT/lower_quality/fadg0/sx379-video-fram1.avi
Found C source: ../Dataset/DeepfakeTIMIT/lower_quality/fadg0/sx289-video-fram1.avi
Found C source: ../Dataset/DeepfakeTIMIT/lower_quality/fadg0/sx199-video-fram1.avi
Found C source: ../Dataset/DeepfakeTIMIT/lower_quality/fadg0/si1279-video-fram1.avi
Found C source: ../Dataset/DeepfakeTIMIT/lower_quality/fadg0/sa2-video-fram1.avi
Found C source: ../Dataset/DeepfakeTIMIT/lower_quality/fadg0/si1909-video-fram1.avi
Found C source: ../Dataset/DeepfakeTIMIT/lower_quality/msjs1/si869-video-mrgg0.avi
Found C source: ../Dataset/DeepfakeTIMIT/lower_quality/msjs1/si1899-video-mrgg0.avi
Found C source: ../Dataset/DeepfakeTIMIT/lower_quality/msjs1/sa1-video-mrgg0.avi
Found C source: ../Dataset/DeepfakeTIMIT/lower_quality/msjs1/sx99-video-mrgg0.avi
Found C s

In [6]:
print(file_name)

['../Dataset/VidTIMIT/mrjo0/sx104.avi', '../Dataset/VidTIMIT/mrjo0/si734.avi', '../Dataset/VidTIMIT/mrjo0/sx14.avi', '../Dataset/VidTIMIT/mrjo0/sa2.avi', '../Dataset/VidTIMIT/mrjo0/sa1.avi', '../Dataset/VidTIMIT/mrjo0/sx374.avi', '../Dataset/VidTIMIT/mrjo0/si1624.avi', '../Dataset/VidTIMIT/mrjo0/si1364.avi', '../Dataset/VidTIMIT/mrjo0/sx194.avi', '../Dataset/VidTIMIT/mrjo0/sx284.avi', '../Dataset/VidTIMIT/mrcz0/sx11.avi', '../Dataset/VidTIMIT/mrcz0/si911.avi', '../Dataset/VidTIMIT/mrcz0/si2171.avi', '../Dataset/VidTIMIT/mrcz0/sx101.avi', '../Dataset/VidTIMIT/mrcz0/sx371.avi', '../Dataset/VidTIMIT/mrcz0/si1541.avi', '../Dataset/VidTIMIT/mrcz0/sa2.avi', '../Dataset/VidTIMIT/mrcz0/sa1.avi', '../Dataset/VidTIMIT/mrcz0/sx191.avi', '../Dataset/VidTIMIT/mrcz0/sx281.avi', '../Dataset/VidTIMIT/mtmr0/sx313.avi', '../Dataset/VidTIMIT/mtmr0/sx403.avi', '../Dataset/VidTIMIT/mtmr0/sa2.avi', '../Dataset/VidTIMIT/mtmr0/sa1.avi', '../Dataset/VidTIMIT/mtmr0/sx43.avi', '../Dataset/VidTIMIT/mtmr0/sx223.av

In [9]:
# reading video fame
# Create a VideoCapture object and read from input file

def extract_multiple_videos(intput_filenames):
    """Extract video files into sequence of images.
       Intput_filenames is a list for video file names"""

    i = 1  # Counter of first video

    # Iterate file names:
    for intput_filename in intput_filenames:
        cap = cv2.VideoCapture(intput_filename)

        # Keep iterating break
        while True:
            ret, frame = cap.read()  # Read frame from first video

            if ret:
                cv2.imwrite(str(i) + '.jpg', frame)  # Write frame to JPEG file (1.jpg, 2.jpg, ...)
                cv2.imshow('frame', frame)  # Display frame for testing
                i += 1 # Advance file counter
            else:
                # Break the interal loop when res status is False.
                break

            cv2.waitKey(100) #Wait 100msec (for debugging)

        cap.release() #Release must be inside the outer loop

In [ ]:
extract_multiple_videos(file_name)